## 原始部分

In [ ]:
from transformers import AutoTokenizer, AutoModel
from peft import PeftModel, PeftConfig
import torch
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


model_name_or_path = "/media/yuanz/新加卷/训练代码/chatglm6b_v2_0716/chatglm2-6b_model"


tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True, device_map='auto', torch_dtype=torch.bfloat16)#.half().cuda()


model = model.eval()

In [ ]:
response, history = model.chat(tokenizer, "你好", history=[])
print(response)
response, history = model.chat(tokenizer, "类型#上衣*材质#牛仔布*颜色#白色*风格#简约*图案#刺绣*衣样式#外套*衣款式#破洞", history=[])
print(response)

## 注意这里需要传递lora训练的路径

In [ ]:
peft_model_id = "output/adgen-chatglm2-6b-lora_version/checkpoint-880"
model = PeftModel.from_pretrained(model, peft_model_id)
model = model.eval()

In [ ]:
response, history = model.chat(tokenizer, "类型#上衣*材质#牛仔布*颜色#白色*风格#简约*图案#刺绣*衣样式#外套*衣款式#破洞", history=[])
print(response)

## 合并模型
1. 很多人合并lora的需求，那么只需要这么做即可

In [ ]:
model_merge = model.merge_and_unload()
merger_lora_model_path = "test_merge_dir"

model_merge.save_pretrained(merge_lora_model_path, max_shard_size="2GB")
tokenizer.save_pretrained(merge_lora_model_path)